# 3 计算图和动态图机制

## 3.1 计算图

### 3.1.1 计算图的定义
深度学习就是对张量进行一系列的操作，随着操作种类和数量的增多，会出现各种值得思考的问题。
<br>比如多个操作之间是否可以并行，如何协同底层的不同设备，如何避免冗余的操作，以实现最高效的计算效率，同时避免一些 bug。
<br>因此产生了计算图 (Computational Graph)。
计算图是用来描述运算的有向无环图，有两个主要元素：节点 (Node) 和边 (Edge)。
<br>节点表示数据，如向量、矩阵、张量。边表示运算，如加减乘除卷积等。
<br>用计算图表示：$y=(x+w)*(w+1)$，如下所示：

![](https://image.zhangxiann.com/20200515221509.png)


## 3.1.2 计算图与梯度求导

上图中$y=(x+w)*(w+1)$可以看作$y=a \times b$ ，其中 $a=x+w$，$b=w+1$。
<br>这里求 $y$ 对 $w$ 的导数：
$\begin{aligned} \frac{\partial y}{\partial w} &=\frac{\partial y}{\partial a} \frac{\partial a}{\partial w}+\frac{\partial y}{\partial b} \frac{\partial b}{\partial w} \ &=b  1+a  1 \ &=b+a \ &=(w+1)+(x+w) \ &=2  w+x+1 \ &=2  1+2+1=5\end{aligned}$
<br>体现到计算图中，就是根节点 $y$ 到叶子节点 $w$ 有两条路径 y -> a -> w和y ->b -> w。根节点依次对每条路径的孩子节点求导，一直到叶子节点w，最后把每条路径的导数相加即可。
![](https://image.zhangxiann.com/20200515221816.png)
<br>代码如下：

In [6]:
import torch
w = torch.tensor([1], requires_grad=True, dtype=float)  # Only Tensors of floating point and complex dtype can require gradients
x = torch.tensor([2], requires_grad=True, dtype=float)  # 因此需要显式设置 dtype=float
a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.multiply(a, b)

y.backward()

print(w.grad)
print(x.grad)

tensor([5.], dtype=torch.float64)
tensor([2.], dtype=torch.float64)


在上面的例子中，$x$ 和 $w$ 是叶子节点，其他所有节点都依赖于叶子节点。
<br>叶子节点的概念主要是为了节省内存，**在计算图中的一轮反向传播结束之后，非叶子节点的梯度是会被释放的**。


In [10]:
print("is_leaf:", w.is_leaf, x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)
print("gradient:", w.grad, x.grad, a.grad, b.grad, y.grad)

is_leaf: True True False False False
gradient: tensor([5.], dtype=torch.float64) tensor([2.], dtype=torch.float64) None None None


## 3.2 PyTorch 的动态图机制
PyTorch 采用的是动态图机制 (Dynamic Computational Graph)，而 Tensorflow 采用的是静态图机制 (Static Computational Graph)。
<br>
<br>动态图是运算和搭建同时进行，也就是可以先计算前面的节点的值，再根据这些值搭建后面的计算图。优点是灵活，易调节，易调试。
<br>PyTorch 里的很多写法跟其他 Python 库的代码的使用方法是完全一致的，没有任何额外的学习成本。
<br>
<br>静态图是先搭建图，然后再输入数据进行运算。
<br>优点是高效，因为静态计算是通过先定义后运行的方式，之后再次运行的时候就不再需要重新构建计算图，所以速度会比动态图更快。但是不灵活。
<br>TensorFlow 每次运行的时候图都是一样的，是不能够改变的，
<br>所以不能直接使用 Python 的 while 循环语句，需要使用辅助函数 tf.while_loop 写成 TensorFlow 内部的形式。